In [1]:
import cv2
import argparse
import numpy as np

In [2]:
def draw_prediction(img, class_id, confidence, x, y, x_plus_w, y_plus_h):

    label = str(classes[class_id])

    color = COLORS[class_id]
    

    cv2.rectangle(img, (x,y), (x_plus_w,y_plus_h), color, 2)

    cv2.putText(img, label +" "+str(round(confidence,2)), (x-10,y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

In [35]:
image = cv2.imread("./example3.jpg")

Width = image.shape[1]
Height = image.shape[0]
scale = 0.00392
classes = None
        

In [36]:
with open("./coco.names",'r') as f:
    classes = [line.strip() for line in f.readlines()]

In [37]:
COLORS = np.random.uniform(0, 255, size=(len(classes), 3))


In [38]:
net = cv2.dnn.readNet("./yolov3.weights","./yolov3.cfg")

In [39]:
blob = cv2.dnn.blobFromImage(image, scale, (416,416), (0,0,0), True, crop=False)

In [40]:
net.setInput(blob)

In [41]:
output_layes_name = net.getUnconnectedOutLayersNames()
outs = net.forward(output_layes_name)

In [42]:
class_ids = []
confidences = []
boxes = []
conf_threshold = 0.5
nms_threshold = 0.4


for out in outs:
    for detection in out:
        scores = detection[5:]
        class_id = np.argmax(scores)
        confidence = scores[class_id]
        if confidence > 0.5:
            center_x = int(detection[0] * Width)
            center_y = int(detection[1] * Height)
            w = int(detection[2] * Width)
            h = int(detection[3] * Height)
            x = center_x - w / 2
            y = center_y - h / 2
            class_ids.append(class_id)
            confidences.append(float(confidence))
            boxes.append([x, y, w, h])


In [43]:
indices = cv2.dnn.NMSBoxes(boxes, confidences, conf_threshold, nms_threshold)

for i in indices:
    
    box = boxes[i]
    x = box[0]
    y = box[1]
    w = box[2]
    h = box[3]
    draw_prediction(image, class_ids[i], confidences[i], round(x), round(y), round(x+w), round(y+h))


In [45]:
cv2.imshow("object detection", image)
cv2.waitKey()

-1

In [65]:
cv2.destroyAllWindows()